In [ ]:
import os
import csv
import sqlite3
import itertools
import time
import math
import collections

import igraph
import scipy.special

import matplotlib.pylab as plt
# %matplotlib widget
%matplotlib inline

In [ ]:
g = igraph.Graph.Read_Pickle(input('Input to read igraph pickle: ').strip())

In [ ]:
stime = time.time()
partition = g.community_leiden(objective_function='modularity')
etime = time.time()
print(f'{len(partition)} during {etime-stime}')

In [ ]:
l = [len(p) for p in partition if len(p) > 9]

In [ ]:
fig1 = plt.figure(figsize=(4, 4))
fig1.set_facecolor('white')
ax1 = fig1.add_subplot()
ax1.hist(l)
ax1.set_title('Histogram of cluster size', fontsize='large')
ax1.set_xlabel('Cluster size', fontsize='x-large')
ax1.set_ylabel('Cluster count', fontsize='x-large')

In [ ]:
# Load database
path = 'file:index.db?mode=ro'
conn = sqlite3.connect(path, uri=True)
cur = conn.cursor()


def get_txin_counts(indices):
    counts = 0
    for n in g.vs.select(indices):
        name = n['name']
        cur.execute('''SELECT COUNT(*) FROM TxIn WHERE addr = ?''', (name,))
        tmp_result = cur.fetchone()
        if tmp_result is not None:
            counts = counts + tmp_result[0]
    return counts


def get_txout_counts(indices):
    counts = 0
    for n in g.vs.select(indices):
        name = n['name']
        cur.execute('''SELECT COUNT(*) FROM TxOut WHERE addr = ?''', (name,))
        tmp_result = cur.fetchone()
        if tmp_result is not None:
            counts = counts + tmp_result[0]
    return counts

In [ ]:
for i in range(0, len(partition)):
    p = partition[i]
    if len(p) == 1:
        continue
    print(f'Cluster Size: {len(p):>5d}', end=' ')
    print(f'TxIn: {get_txin_counts(p) / len(p):13.10f}', end=' ')
    print(f'TxOut: {get_txout_counts(p) / len(p):13.10f}')

In [ ]:
stime = time.time()
rank = g.pagerank()
etime = time.time()
print(f'Max pagerank: {max(rank)} at {g.vs[rank.index(max(rank))]["name"]} during {etime-stime}')

In [ ]:
stime = time.time()
g2 = partition.cluster_graph(combine_vertices='random', combine_edges='sum')
g2.delete_vertices(g2.vs.select(_degree=0))
layout = g2.layout_drl()
igraph.plot(g2, 'partition.svg', layout=layout, vertex_size=5, edge_width=0.5, edge_color='red')
etime = time.time()
print(f'Plotting done during {etime-stime}')